# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\dugga\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA

parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))

C:\Users\dugga\AppData\Local\Temp\ipykernel_9464\2690469203.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [6]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
LVS,2021-01-04,59.799999,60.290001,57.320000,57.950001,57.041168,10089600,Consumer Discretionary,Casinos & Gaming,2021,NaN,NaN,2.970001
LVS,2021-01-05,57.790001,59.689999,57.720001,59.439999,58.507801,2879200,Consumer Discretionary,Casinos & Gaming,2021,57.950001,0.025712,1.969997
LVS,2021-01-06,59.049999,60.250000,57.980000,58.540001,57.621918,4055100,Consumer Discretionary,Casinos & Gaming,2021,59.439999,-0.015141,2.270000
LVS,2021-01-07,58.500000,58.930000,57.630001,58.150002,57.238037,4685700,Consumer Discretionary,Casinos & Gaming,2021,58.540001,-0.006662,1.299999
LVS,2021-01-08,58.279999,59.790001,57.009998,57.919998,57.011642,6859900,Consumer Discretionary,Casinos & Gaming,2021,58.150002,-0.003955,2.780003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GILD,2019-12-24,66.809998,66.940002,66.290001,66.720001,55.365940,1965800,Health Care,Biotechnology,2019,66.750000,-0.000449,0.650002
GILD,2019-12-26,66.709999,66.910004,66.150002,66.389999,55.092087,3527400,Health Care,Biotechnology,2019,66.720001,-0.004946,0.760002
GILD,2019-12-27,66.379997,66.489998,65.709999,65.949997,54.726948,4463900,Health Care,Biotechnology,2019,66.389999,-0.006628,0.779999


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [7]:
# Write your code below.
dd_feat = dd_feat.compute()
dd_feat['10dayrollingaverage'] = dd_feat['returns'].rolling(10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it is not necessary to convert to a pandas DataFrame to calculate the moving average return.



+ Would it have been better to do it in Dask? Why?
Yes, using Dask would have been better due to its ability to handle large datasets efficiently and its parallel processing capabilities





(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.